# Elasticity equation

In this tutorial we present how to solve the elasticity equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unknown is the displacement $u$.

Let $\Omega$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$\lambda$ Lamé constant and $\mu$ the Kirchhoff modulus, we want to solve the following problem: find $u$ such that
$$
\nabla \cdot [ 2 \mu \epsilon(u) + \lambda \nabla \cdot u] = -b
$$
with $\epsilon$ the symmetric gradient and $b$ a body force. The stress tensor, which can be post-processed from $u$, is given by
$$
    \sigma = 2 \mu \epsilon(u) + \lambda \nabla \cdot u I
$$


## Exercise 3: footing problem - locking effects

The footing problem is when a force is impose on the top compressing the body and the bottom is fixed.

For this test case we set $\Omega = [0, 1]^2$, $b = 0$, and the following boundary conditions:
$$ 
u = 0 \text{ on } \partial_{bottom} \Omega \qquad \nu \cdot \sigma = 0 \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega \qquad \nu \cdot \sigma = [0, -1e-3]^\top \text{ on } \partial_{top} \Omega
$$

For $\nu \rightarrow 0.5$ the material becomes incompressible and the numerical solution may suffer from locking effects. We have the following relations between the $\lambda$ and $\mu$ as functions of the Young's modulus $E$ and Poisson ratio $\nu$
$$
\lambda = \dfrac{\nu E}{(1+\nu)(1-2\nu)}\qquad \mu = \dfrac{E}{2(1+\nu)}
$$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
import numpy as np

import porepy as pp
import pygeon as pg

We create now the grid, since we use a vector Lagrangian of order 1 for ${u}$ we are restricted to simplices. In this example we consider a 2-dimensional structured grid, but the presented code will work also in 3d.

In [2]:
N = 10
dim = 2
mesh_size = 1 / N

structured = True  # True False
sd = pg.unit_grid(dim, mesh_size, as_mdg=False, structured=structured)

sd.compute_geometry()

With the following code we set the data, in particular the Lamé and the Kirchhoff modulus, and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [3]:
key = "elasticity"

E = 1
nu = 0.25  # 0.1 0.25 0.4999

lambda_ = E * nu / ((1 + nu) * (1 - 2 * nu))
mu = E / (2 * (1 + nu))

print("Lambda and mu", lambda_, mu)

bottom = np.hstack([np.isclose(sd.nodes[1, :], 0)] * dim)
top = np.isclose(sd.face_centers[1, :], 1)

fun = lambda _: np.array([0, -1e-3])

data = {pp.PARAMETERS: {key: {"lambda": lambda_, "mu": mu}}}

Lambda and mu 0.4 0.4


Once the data are assigned to the grid, we construct the matrices. Once the latter is created, we also construct the right-hand side containing the boundary conditions.

In [4]:
vec_p1 = pg.VecLagrange1(key)

# we construct the matrix
A = vec_p1.assemble_stiff_matrix(sd, data)
# we compute the boundary conditions
b = vec_p1.assemble_nat_bc(sd, fun, top)

We need to solve the linear system, PyGeoN provides a framework for that. The actual imposition of essential boundary conditions (displacement boundary conditions) might change the symmetry of the global system, the class `pg.LinearSystem` preserves this structure by internally eliminating these degrees of freedom.

In [5]:
ls = pg.LinearSystem(A, b)
ls.flag_ess_bc(bottom, np.zeros(vec_p1.ndof(sd)))
u = ls.solve()

Since the computed $u$ is a vector per peak of the grid, for visualization purposes we project the displacement in each cell center as vector. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [6]:
# we need to add the z component for the exporting
u = np.hstack((u, np.zeros(sd.num_nodes))).reshape((3, -1))

save = pp.Exporter(sd, "sol", folder_name="ex3")
save.write_vtu(data_pt=[("u", u)])

In [9]:
# Consistency check
assert np.isclose(np.linalg.norm(u), 0.00603326781876562)